# Classificando o dataset MNIST usando MLPs e CNNs

### Membros

* Gabriel Pessoa
* Ícaro Guerra
* Lucas Barros
* Matheus Pessoa
* Rafael Mota

## Introdução
Esse relatório detalha o processo experimental para o desenvolvimento de uma solução para o problema de classificação de dígitos escritos manualmente do dataset MNIST usando Redes Neurais dos tipos: Multilayer Perceptron (MLP) e Convolutional Neural Network (CNN). O dataset MNIST consiste em 70 mil imagens 28x28 dos dígitos de 0 a 9, sendo 60 mil samples de treinamento e 10 mil samples de teste.

## Bibliotecas Utilizadas
Para a implementação utilzaremos as seguintes bibliotecas:


In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

* O Numpy e o Pandas são usados para a representação dos dados e para a implementação de funções auxiliares.

* Tensorflow e Keras são usados para a implementação e treinamento das Redes Neurais.

## Carregamento dos Dados
O código a seguir carrega os dados do dataset, adapta ao formato desejado e define os parâmetros globais sobre os dados.

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Parâmetros de Treinamento
Aqui definimos os parâmetro de treinamento que serão utilizados para todas as redes neurais, a escolha dessa quantidade de epochs e batch_size, foi feita para que tenhamos maior perfomance e mais dinamicidade no experimento com os treinamentos.

In [3]:
# Training parameters
epochs = 15
batch_size = 128
metrics = ["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]

## Funções de Avaliação
Aqui definimos a duas funções que vamos usar para avaliar as soluções, a primeira função `accuracy_per_class` retorna a métrica de *acurácia* para cada uma das classes de resposta, que no nosso caso são os possíveis dígitos de 0 a 9. A segunda função `accuracy_precision_recall` retorna as métricas *acurácia*, *precision* e *recall* total.

Um breve descrição das métricas citadas seria:

* *acurácia*: A proporção de predição corretas com o total de casos.
* *precision*: A proporção de predições positivas corretas com o total de predições positivas da classe.
* *recall*: A proporção de predições positivas corretas com o total de casos da classe.

In [4]:
def accuracy_per_class(model):
    y_pred = model.predict(x_test)

    correct = [0] * num_classes
    total = [0] * num_classes

    for real, pred in zip(y_test.argmax(axis=1), y_pred.argmax(axis=1)):
        if real == pred:
            correct[real] += 1
        total[real] += 1

    accuracies = []
    for correct, total in zip(correct, total):
        accuracies.append(correct / total)

    return pd.Series(data=accuracies, index=range(0,num_classes), name="Accuracy per class")

def accuracy_precision_recall(model):
    _, acc, prec, rec = model.evaluate(x_test, y_test,verbose=0)

    return pd.Series(data=[acc, prec, rec],index=["Accuracy", "Precision", "Recall"])

In [5]:
model = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Flatten(),
    layers.Dense(50),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=metrics)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                39250     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________


In [6]:

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/15
422/422 [==============================] - 3s 4ms/step - loss: 0.4718 - accuracy: 0.8682 - precision: 0.9321 - recall: 0.7972 - val_loss: 0.2539 - val_accuracy: 0.9302 - val_precision: 0.9499 - val_recall: 0.9098
Epoch 2/15
422/422 [==============================] - 2s 4ms/step - loss: 0.3008 - accuracy: 0.9158 - precision: 0.9366 - recall: 0.8981 - val_loss: 0.2404 - val_accuracy: 0.9340 - val_precision: 0.9513 - val_recall: 0.9190
Epoch 3/15
422/422 [==============================] - 2s 5ms/step - loss: 0.2861 - accuracy: 0.9197 - precision: 0.9367 - recall: 0.9055 - val_loss: 0.2353 - val_accuracy: 0.9332 - val_precision: 0.9482 - val_recall: 0.9237
Epoch 4/15
422/422 [==============================] - 2s 5ms/step - loss: 0.2772 - accuracy: 0.9231 - precision: 0.9390 - recall: 0.9097 - val_loss: 0.2281 - val_accuracy: 0.9377 - val_precision: 0.9512 - val_recall: 0.9252
Epoch 5/15
422/422 [==============================] - 1s 3ms/step - loss: 0.2715 - accuracy: 0.9236 - pr

In [7]:
accuracy_per_class(model)

0    0.974490
1    0.977093
2    0.898256
3    0.913861
4    0.913442
5    0.860987
6    0.954071
7    0.934825
8    0.902464
9    0.916749
Name: Accuracy per class, dtype: float64

In [8]:
accuracy_precision_recall(model)

Accuracy     0.925800
Precision    0.937225
Recall       0.916700
dtype: float64